In [1]:
%load_ext dotenv
%dotenv

In [12]:
import logging
import os
from solana.rpc.async_api import AsyncClient
from zeta_py.types import Network, Asset, Side, OrderOptions, OrderType
from solana.rpc.commitment import Confirmed
from solana.rpc.types import TxOpts
from zeta_py import utils
from zeta_py.client import Client
import anchorpy
from datetime import datetime, timedelta

logger = logging.getLogger("zeta_py")
logging.basicConfig(level=logging.DEBUG)

endpoint = os.getenv("ENDPOINT", "https://api.mainnet-beta.solana.com")
print(endpoint.split(".com")[0])
conn = AsyncClient(endpoint, commitment=Confirmed, blockhash_cache=False)

wallet = anchorpy.Wallet.local()  # get local filesystem keypair wallet at ~/.config/solana/id.json
# print(wallet.public_key)

https://mainnet.helius-rpc


In [3]:
tx_opts = TxOpts({"preflight_commitment": Confirmed, "skip_confirmation": False})

client = await Client.load(Network.MAINNET, conn, wallet, [Asset.SOL], subscribe=False, tx_opts=tx_opts)

INFO:zeta_py.accounts.Account:Loaded account: State
INFO:zeta_py.accounts.Account:Loaded account: Pricing
INFO:zeta_py.accounts.Account:Loaded account: Clock
INFO:zeta_py.accounts.Account:Subscribed to Clock
INFO:zeta_py.accounts.Account:Loaded account: CrossMarginAccount


### Client state

In [4]:
print(client.balance)
print(client.positions)
print(client.open_orders)

0.294042
{<Asset.SOL: 0>: Position(size=0.002, cost_of_trades=0.038354)}
{<Asset.SOL: 0>: [Order(order_id=1844692854115028760145792, client_id=0, open_order_address=Pubkey(
    ciwBQfLdM97vFp8GwYYLczPgYoLfdkTVD6dhHAeGoWj,
), open_order_slot=0, fee_tier=0, info=OrderInfo(price=0.1, size=0.001, price_lots=100000, size_lots=1), side=<Side.Bid: 0>, tif_offset=0)]}


### Deposit
Deposit USDC into your Zeta margin account

In [5]:
await client.deposit(0.1)

INFO:zeta_py.accounts.Account:Loaded account: CrossMarginAccountManager
INFO:zeta_py.client.Client:Depositing 0.1 USDC to margin account


Signature(
    3qvyNjHMdw6a21fPTmjqAPYpHGFJAHbxVgyvcL1s2AQTys9jpzgK5rY2gYfcNwo7781rH8eQMUDuKGa6XjkyEB4C,
)

### Place Order

In [6]:
await client.place_order(asset=Asset.SOL, price=0.1, size=0.001, side=Side.Bid)

INFO:zeta_py.client.Client:Placed 0.001x SOL-PERP Bid @ $0.1


Signature(
    2bPeZMb4MEv2WDbfRFhNMkVnWeXb2xUFFkBZ2qTRmwHtpMGXfXjS2Wmf6FXSEkja3RFaMhCb2bKdzDoHfp3XFvj6,
)

### Cancel Order

In [10]:
open_orders = await client.fetch_open_orders(Asset.SOL)
print("Open Orders:")
for order in open_orders:
    print(f"{order.side.name} {order.info.size}x ${order.info.price}")

Open Orders:
Bid 0.001x $0.1


In [9]:
oid = open_orders[0].order_id
await client.cancel_order(Asset.SOL, order_id=oid, side=Side.Bid)

INFO:zeta_py.client.Client:Cancelling order 1844692854115028760144731 for SOL


Signature(
    GdPeAApEte86HLJGJLTHyQZBC8kCAUtDnhXXDzYH2qDwnwkzCfuf1ofQ9zXaP9phrcC3QBdkW6mbwgbG9mw39Kf,
)

### Quoting
Use this code to atomically cancel and replace a set of quotes on Zeta.

*For this example we quote 0.001 SOL-PERP contracts at 20bps spread*

In [7]:
# Quoting parameters
asset = Asset.SOL  # SOL-PERP market
quote_size = 0.001  # min size
quote_bps = 10  # basis points from midpoint
time_in_force = 10  # expire the quote after 10 seconds

# Get the latest mark price and calculate bid/ask prices
price_fixed = client.exchange.pricing.account.mark_prices[asset.to_index()]
bid_price = utils.convert_fixed_int_to_decimal(price_fixed * (1 - quote_bps / 10000))
ask_price = utils.convert_fixed_int_to_decimal(price_fixed * (1 + quote_bps / 10000))

# Set order options
expiry_ts = int((datetime.now() + timedelta(seconds=time_in_force)).timestamp())
order_opts = OrderOptions(expiry_ts=expiry_ts, order_type=OrderType.Limit, client_order_id=123)

# Execute quote!
await client.replace_quote(Asset.SOL, bid_price, quote_size, ask_price, quote_size, order_opts)

INFO:zeta_py.client.Client:Replacing SOL orders: 0.001x Bid @ $21.757750469999998, 0.001x Ask @ $21.801309529999997


Signature(
    3Lb5vqF3pQKSqXBLm4hx4EMgRtMJctcSuwgZ8unRfXF9wgFewQuXF7JMTpgDDURQMcPMfMLy1XifVD9P6o6KgMDd,
)

## Examples
### Simple MM Bot
Takes the midpoint price from Binance USDT margined futures market and quotes it on Zeta.

In [16]:
import ccxt.async_support as ccxt

async def quote(client, asset: Asset, price):
    # Quoting parameters
    quote_size = 0.001  # min size
    quote_bps = 10  # basis points from midpoint
    time_in_force = 60  # expire the quote after 60 seconds

    # Get the latest mark price and calculate bid/ask prices
    # price_fixed = client.exchange.pricing.account.mark_prices[asset.to_index()]
    bid_price = price * (1 - quote_bps / 10000)
    ask_price = price * (1 + quote_bps / 10000)

    # Set order options
    expiry_ts = int((datetime.now() + timedelta(seconds=time_in_force)).timestamp())
    order_opts = OrderOptions(expiry_ts=expiry_ts, order_type=OrderType.PostOnlySlide, client_order_id=123)

    # Execute quote!
    await client.replace_quote(asset, bid_price, quote_size, ask_price, quote_size, order_opts)
    
async def get_binance_midpoint(exchange, symbol):
    # Slight annoyance that CCXT returns SOL price to 2dp whereas it should be 3dp
    orderbook = await exchange.fetch_order_book(symbol, limit=1)
    return (orderbook['bids'][0][0]+orderbook['asks'][0][0])/2

async def naive_quoter(symbol):
    # you can set enableRateLimit = True to enable the built-in rate limiter
    # this way you request rate will never hit the limit of an exchange
    # the library will throttle your requests to avoid that

    exchange = ccxt.binance()
    mid_price = await get_binance_midpoint(exchange, symbol)
    await quote(client, Asset.SOL, mid_price)
    while True:
        print('-----------------------------------------------------------------')
        print(exchange.iso8601(exchange.milliseconds()), 'fetching', symbol, 'orderbook from', exchange.name)
        try:
            new_mid_price = await get_binance_midpoint(exchange, symbol)
            print(f'Binance midpoint: {new_mid_price}')
            if new_mid_price != mid_price:
                mid_price = new_mid_price
                print(f'Binance price moved to {mid_price}, requoting on Zeta...')
                await quote(client, Asset.SOL, mid_price)
            
        except ccxt.RequestTimeout as e:
            print('[' + type(e).__name__ + ']')
            print(str(e)[0:200])
            # will retry
        except ccxt.DDoSProtection as e:
            print('[' + type(e).__name__ + ']')
            print(str(e.args)[0:200])
            # will retry
        except ccxt.ExchangeNotAvailable as e:
            print('[' + type(e).__name__ + ']')
            print(str(e.args)[0:200])
            # will retry
        except ccxt.ExchangeError as e:
            print('[' + type(e).__name__ + ']')
            print(str(e)[0:200])
            break  # won't retry
          
symbol = "SOL/USDT"
await naive_quoter(symbol)

INFO:zeta_py.client.Client:Replacing SOL orders:                 0.001x Bid @ $19.255724999999998, 0.001x Ask @ $19.294274999999995


-----------------------------------------------------------------
2023-09-05T04:51:19.993Z fetching SOL/USDT orderbook from Binance
Binance midpoint: 19.275
-----------------------------------------------------------------
2023-09-05T04:51:20.104Z fetching SOL/USDT orderbook from Binance
Binance midpoint: 19.275
-----------------------------------------------------------------
2023-09-05T04:51:20.208Z fetching SOL/USDT orderbook from Binance
Binance midpoint: 19.275
-----------------------------------------------------------------
2023-09-05T04:51:20.320Z fetching SOL/USDT orderbook from Binance
Binance midpoint: 19.275
-----------------------------------------------------------------
2023-09-05T04:51:20.426Z fetching SOL/USDT orderbook from Binance
Binance midpoint: 19.275
-----------------------------------------------------------------
2023-09-05T04:51:20.541Z fetching SOL/USDT orderbook from Binance
Binance midpoint: 19.275
----------------------------------------------------------

INFO:zeta_py.client.Client:Replacing SOL orders:                 0.001x Bid @ $19.265715, 0.001x Ask @ $19.304284999999997


Binance midpoint: 19.275
-----------------------------------------------------------------
2023-09-05T04:52:27.910Z fetching SOL/USDT orderbook from Binance
Binance midpoint: 19.285
Binance price moved to 19.285, requoting on Zeta...
-----------------------------------------------------------------
2023-09-05T04:52:29.258Z fetching SOL/USDT orderbook from Binance
Binance midpoint: 19.285
-----------------------------------------------------------------
2023-09-05T04:52:29.372Z fetching SOL/USDT orderbook from Binance
Binance midpoint: 19.285
-----------------------------------------------------------------
2023-09-05T04:52:29.475Z fetching SOL/USDT orderbook from Binance
Binance midpoint: 19.285
-----------------------------------------------------------------
2023-09-05T04:52:29.589Z fetching SOL/USDT orderbook from Binance
Binance midpoint: 19.285
-----------------------------------------------------------------
2023-09-05T04:52:29.693Z fetching SOL/USDT orderbook from Binance
Binanc

CancelledError: 